# Analyse evaluation scores (BERT & ROGUE-L)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("results/evaluation_scores.csv")

In [3]:
df

,summary,dialog,system_prompt,messages,text,prompt,summary_by_SmolLM2-360M-Instruct-summarizer,summary_by_SmolLM2-1.7B-Instruct-summarizer,summary_by_SmolLM2-1.7B-Instruct,summary_by_SmolLM2-360M-Instruct,bert_score_summary_by_SmolLM2-360M-Instruct-summarizer,rougeL_score_summary_by_SmolLM2-360M-Instruct-summarizer,bert_score_summary_by_SmolLM2-1.7B-Instruct-summarizer,rougeL_score_summary_by_SmolLM2-1.7B-Instruct-summarizer,bert_score_summary_by_SmolLM2-1.7B-Instruct,rougeL_score_summary_by_SmolLM2-1.7B-Instruct,bert_score_summary_by_SmolLM2-360M-Instruct,rougeL_score_summary_by_SmolLM2-360M-Instruct
0,The conversation centers around Liam Howlett's...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'You are an expert in summarizing...,The conversation centered around the musical c...,The conversation centered around the musical c...,The conversation centered around the musical c...,The conversation centered around the musical c...,0.441898,0.333333,0.441898,0.333333,0.441898,0.333333,0.441898,0.333333
1,The speakers discuss their experiences working...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'You are an expert in summarizing...,A group of nurses discuss their experiences wo...,A group of nurses discuss their experiences wo...,A group of nurses discuss their experiences wo...,A group of nurses discuss their experiences wo...,0.427690,0.341463,0.427690,0.341463,0.427690,0.341463,0.427690,0.341463
2,This conversation revolves around a group of d...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'You are an expert in summarizing...,"A group of doctors discuss various topics, inc...","A group of doctors discuss various topics, inc...","A group of doctors discuss various topics, inc...","A group of doctors discuss various topics, inc...",0.647711,0.666667,0.647711,0.666667,0.647711,0.666667,0.647711,0.666667
3,The speakers discuss attending a meeting of th...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'You are an expert in summarizing...,The conversation centers around a planned Astr...,The conversation centers around a planned Astr...,The conversation centers around a planned Astr...,The conversation centers around a planned Astr...,0.571095,0.360000,0.571095,0.360000,0.571095,0.360000,0.571095,0.360000
4,"A group discusses a closed bakery, an upcoming...",Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'You are an expert in summarizing...,The speakers discuss the closure of a bakery o...,The speakers discuss the closure of a bakery o...,The speakers discuss the closure of a bakery o...,The speakers discuss the closure of a bakery o...,0.688221,0.575758,0.688221,0.575758,0.688221,0.575758,0.688221,0.575758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,A conversation took place about Benford's Law ...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'You are an expert in summarizing...,"The conversation discusses Benford's Law, its ...","The conversation discusses Benford's Law, its ...","The conversation discusse

In [11]:
identicals = []
for i, row in df.iterrows():
    if len(
        set(
            [row["summary_by_SmolLM2-360M-Instruct-summarizer"], 
             row["summary_by_SmolLM2-360M-Instruct"], 
             row["summary_by_SmolLM2-1.7B-Instruct"], 
             row["summary_by_SmolLM2-1.7B-Instruct-summarizer"]]
             )
    ) == 1:
        identicals.append(True)
    else:
        identicals.append(False)
        
    #print(row["summary_by_SmolLM2-360M-Instruct-summarizer"])
    #print(row["summary_by_SmolLM2-360M-Instruct-summarizer"])

In [12]:
all(identicals)

True

In [9]:
ls = ["The conversation. Is center", "The conversation. Is center", "The Conversation. Is center"]
len(set(ls))

2